In [ ]:
import numpy as np
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from classes.diya_communicator import Communicator

diya_name = "07"
comm = Communicator(diya_name, save_data=False, save_frequency=10, choose_specific_directory=False, verbose=False)

In [ ]:
from controllers.PID_controller import PIDController
from scipy.constants import convert_temperature as conv_temp

dt_disc = 0.0 # s
kp, ki, kd = -2.5, -4.0, 0.0
setpoint = conv_temp(23.0, 'C', 'K') # K
output_limits = (-3, 3) # A
controller = PIDController(kp, ki, kd, setpoint, dt_disc, output_limits)

In [ ]:
import datetime
from classes.LED_params import LEDparams
from classes.model import Model
# from classes.mqtt_listener import MQTTListener

LEDpar = LEDparams()
LEDpar.program_reader()

# Initial conditions
T_amb0 = conv_temp(25.0, 'C', 'K') # K
x0 = np.array([0.9, T_amb0, T_amb0])

model = Model(LEDpar, x0, T_amb0=T_amb0)

R4 = model.R_4_lambda
R5 = model.R_5
frac = R5/(R4 + R5)

x_FAN = 0.0

duration = 6 # seconds
T_control = dt_disc # seconds
T_print = 2 # seconds

continue_loop = True
initial_time = datetime.datetime.now()
last_print_time = initial_time
last_control_time = initial_time

try:
    comm.start()

    while continue_loop:
        current_time = datetime.datetime.now()
        elapsed_time = (current_time - initial_time).total_seconds()

        if (current_time - last_print_time).total_seconds() >= T_print:
            print(comm.last_measurement)
            last_print_time = current_time

        # Control action every T_control seconds
        if (current_time - last_control_time).total_seconds() >= T_control:

            meas_df = comm.last_measurement
            Tc = (meas_df["T1_C"].iloc[0] + meas_df["T2_C"].iloc[0])/2
            Tc_K = conv_temp(Tc, 'C', 'K')
            Tcell_K = frac*Tc_K + (1 - frac)*T_amb0
            u = controller.get_control_input(Tcell_K)
            print(u[0])
            comm.send_control_input(u[0], x_FAN)

            last_control_time = current_time

        if elapsed_time > duration:
            continue_loop = False

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    comm.stop()

Communicator started
   T1_C  T2_C  T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
0    25    25    25     0      0        0        0
   T1_C  T2_C  T3_C  x_HP  x_FAN  U_HP_mV  I_HP_mA
0    25    25    25     0      0        0        0
Communicator stopped


In [4]:
# import paho.mqtt.client as mqtt

# # Define the MQTT broker details
# broker = "mqtt.119.ovh"
# port = 1883
# topic = "diya06"
# read_topic = topic + "/rx/"
# transmit_topic = topic + "/tx/"
# keep_alive = 60

# last_message = None

# # Callback function when a message is received
# def on_message(client, userdata, message):
#     last_message = message.payload.decode()
#     # print(f"Message received: {last_message} on topic {message.topic}")

#     parts = last_message.split()
#     if "TMP&U" not in last_message or "Peltier:" not in last_message or "Fan:" not in last_message:
#         print("Received incomplete message")
#         return
    
#     # Parse each component based on its position in the message
#     T_1 = float(parts[2])
#     T_2 = float(parts[3])
#     T_3 = float(parts[4])

#     # Parse the 'Peltier' and 'Fan' values by removing their labels
#     peltier_value = int(parts[6])
#     fan_value = int(parts[8])

#     # Print out parsed values
#     print("Measurements:", T_1, T_2, T_3)
#     print("Peltier Value:", peltier_value)
#     print("Fan Value:", fan_value)
#     print("-------------------------------------------")

# def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
#     sign_HP = "+" if x_HP >= 0 else "-"

#     # Bound control values
#     x_HP = max(min(x_HP, 100), -100)
#     x_FAN = max(min(x_FAN, 100), 0)

#     # Format control values
#     x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
#     x_FAN_formatted = f"{int(x_FAN):03d}"

#     message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

#     client.publish(transmit_topic, message_str)
#     print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# # Create an MQTT client instance
# # cself.client = mqtt.Client(api_version=mqtt.CallbackAPIVersion.VERSION2) ##################################3
# client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# # Assign the on_message callback
# client.on_message = on_message

# # Connect to the broker
# client.connect(broker, port, keep_alive)

# # Subscribe to the specified topic
# client.subscribe(read_topic)

In [5]:
# I_HP = 0
# x_FAN = 0

# send_control_input(client, I_HP, x_FAN)

# client.loop_start()

In [6]:
# client.loop_stop()

In [7]:
# client.disconnect()